In [9]:
!pip install scrapy
!pip install crochet

In [10]:
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['https://quotes.toscrape.com']

    def parse(self, response):
        page = response.css('div.quote')
        for element in page:
            text = element.css("span.text::text").get()
            yield { 'text': text }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            self.log(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [11]:
import crochet
from scrapy.crawler import CrawlerProcess

crochet.setup()

@crochet.run_in_reactor
def run_spider():
    process = CrawlerProcess(settings={'FEEDS': {'output.json': {'format': 'json'}}})
    process.crawl(QuotesSpider)
    process.start()

run_spider()

Exception in thread CrochetReactor:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/crochet/_eventloop.py", line 360, in <lambda>
    target=lambda: self._reactor.run(installSignalHandlers=False),
  File "/usr/local/lib/python3.10/dist-packages/twisted/internet/base.py", line 695, in run
    self.startRunning(installSignalHandlers=installSignalHandlers)
  File "/usr/local/lib/python3.10/dist-packages/twisted/internet/base.py", line 926, in startRunning
    raise error.ReactorNotRestartable()
twisted.internet.error.ReactorNotRestartable
